In [1]:
import time
import random
import numpy as np
from collections import defaultdict
from optparse import OptionParser
import pickle
import scipy.misc as smi
from PIL import Image

# Required libraries
import h5py
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.layers.normalization import BatchNormalization as BN

import matplotlib.pylab as plt
from IPython.display import Image, display
%matplotlib inline

import sys
sys.path.append('../')
# d-script imports
from data_iters.minibatcher import MiniBatcher
from data_iters.iam_hdf5_iterator import IAM_MiniBatcher
from viz_tools.array_to_png import get_png_from_array, display_img_array

from fielutil import *

Using Theano backend.
Using gpu device 0: GeForce GTX 980M (CNMeM is disabled)


In [ ]:
print "Loading original weights into GPU memory"
model = fielnet('../convnets/fielnet/fielnet.hdf5', layer='fc8')
model.compile(loss='mse', optimizer='sgd')
print "Finished weight load"
input_shape = model.input_shape
output_shape = model.output_shape[1]

In [ ]:
im = Image.open('/fileserver/nmec-handwriting/stil-writing-corpus-processed/binary_halfims/FR-003-003.bin.tif')
im = np.array(im).astype(np.int)
plt.imshow(im, cmap='gray')

In [ ]:
im = halfim[0]

vskip = 120
hskip = 120

obufx = (im.shape[1]-input_shape[2])/vskip+1
obufy = (im.shape[2]-input_shape[3])/hskip+1
obuflen = obufx*obufy
ibuflen = 3200

# input_buffer = np.zeros((obuflen, 1, input_shape[2], input_shape[3]))
input_buffer = np.zeros((ibuflen, 1, input_shape[2], input_shape[3]))
output_buffer = np.zeros( (obuflen, output_shape) )

In [ ]:
import sys
print "Begin data transfer to buffers"
k = 0; lb=0; le=0
for i in xrange(0, im.shape[1] - input_shape[2] + vskip, vskip):
    for j in xrange(0, im.shape[2] - input_shape[3] + hskip, hskip):
        input_buffer[k] = im[:, i:(i+input_shape[2]), j:(j+input_shape[3])]
        k+=1; le+=1;
        # Dump buffer and do inference
        if k==ibuflen:
            output_buffer[lb:le] = model.predict( input_buffer )
            pctdone = float(le)/obuflen*100
            sys.stdout.write("\033[F") # Cursor up one line
            print ("Finished output_buffer fill (%d - %d) / %d : %f %%" %( lb, le, obuflen, float(le)/obuflen*100 ))
            k=0; lb=le

In [ ]:
output_map = output_buffer.reshape(obufx, obufy, output_shape)

In [ ]:
numcols = 3
numrows = 16

k = 0
for j in xrange(numrows):
    plt.figure()
    for i in xrange(numcols):
        if k < output_shape:
            plt.subplot(1,numcols,i+1)
            plt.imshow(output_map[:,:,k]) #,aspect='auto')
            plt.title('Neuron '+str(j*numcols+i))
            # plt.colorbar()

In [ ]:
import pylab
import matplotlib.cm as cm
k = 0
for j in xrange(numrows):
    f = pylab.figure()
    for i in xrange(numcols):
        if k < output_shape:
            f.add_subplot(1, numcols, i+1)
            pylab.imshow(output_map[:,:,k],cmap=cm.Greys_r)
            k+=1